In [110]:
#Amazon review label 2 is positive, label 1 is negative
import pandas as pd
ignore_top_N_stopwords= 2
ignore_word_with_less_than_M_occurences = 2

train = pd.read_csv("small_train.csv",header=None, delimiter=',',names=
                   ["TARGET","TITLE","REVIEW"])
test = pd.read_csv("small_test.csv",header=None, delimiter=',',names=
                   ["TARGET","TITLE","REVIEW"])
train_with_no_targets = train[["REVIEW"]]
test_with_no_targets = test[["REVIEW"]]
train_targets = train[["TARGET"]]
test_targets = test[["TARGET"]]
#test.head()
dfs = [train_with_no_targets, test_with_no_targets]
combined = pd.concat(dfs)
#combined.head(10)
combined_as_list = [item for sublist in combined.values.tolist() for item in sublist]
combined_as_list_of_lists = [x.split(" ") for x in combined_as_list]
LoW = [item.lower() for sublist in combined_as_list_of_lists for item in sublist]
SoW = set(LoW)
#print(len(combined_as_list_of_lists))
#print(len(train_targets.values.tolist()))
#print(len(test_targets.values.tolist()))


In [111]:
##create dict/list of lists with word count, trimmed stopwords and rare words
FinList = []
for word in SoW:
	counter = 0
	for word1 in LoW:
		if word1 == word:
			counter = counter + 1
	FinList.append([counter, word])
FinList.sort(reverse=True)
FinFinList = FinList[ignore_top_N_stopwords:]
FINAL = [x for x in FinFinList if x[0]>ignore_word_with_less_than_M_occurences]
#FINAL

In [112]:
FINAL2 = [x[1] for x in FINAL]
FINAL2.insert(0,"TARGET")
FINAL3 = [x[1] for x in FINAL] #final3 is the same as final2 but does not have TARGET
BoW_CSV = [FINAL2] # first row of the BoWCSV
#len(FINAL3)

In [115]:
#print(combined_as_list_of_lists[:2])
CLL=[]
for fragment in combined_as_list_of_lists:
    newFragment=[]
    for word in fragment:
        if word.lower() in FINAL3:
            newWord=word.lower()
            newFragment.append(newWord)
    CLL.append(newFragment)
    
#print(CLL[:2])
tr_lista=[]            
for i in range(len(train_targets)):
    new_row = train_targets.values.tolist()[i]
    for WORD in FINAL3:
        if WORD in CLL[i]:
            new_row.append(1)
            
        else:
            new_row.append(0)
    assert len(new_row)==len(FINAL2)
    #print(new_row)    
    tr_lista.append(new_row)
tr_headers = FINAL2
tr_BoW = pd.DataFrame(tr_lista, columns=tr_headers)



te_lista=[]            
for i in range(len(test_targets)):
    new_row = test_targets.values.tolist()[i]
    for WORD in FINAL3:
        if WORD in CLL[i+len(train_targets)]:
            new_row.append(1)
            
        else:
            new_row.append(0)
    assert len(new_row)==len(FINAL2)
    #print(new_row)    
    te_lista.append(new_row)
te_headers = FINAL2
te_BoW = pd.DataFrame(te_lista, columns=te_headers)
#print(len(te_headers))

te_BoW

#tr_BoW
te_BoW.to_csv("test_BoW.csv",index=False)
tr_BoW.to_csv("train_BoW.csv",index=False)